<a href="https://colab.research.google.com/github/park-geun-hyeong/practice_pytorch/blob/main/asl_dataset_ConvNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. Data preparation

In [3]:
import pandas as pd
import numpy as np
from PIL import Image

In [21]:
import torchvision.transforms as transforms
transform = transforms.Compose([
                                transforms.Resize(32),
                                transforms.RandomCrop(28)
])

In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
device = torch.device("cuda:0")

In [57]:
class ASL_Dataset(Dataset):
    def __init__(self, path, transform=None):
        super(ASL_Dataset, self).__init__()

        train_df = pd.read_csv(path)
        y_train = train_df['label']
        del train_df['label']
        x_train = train_df.values

        self.x_train = x_train
        self.y_train = y_train
        self.transform = transform


    def __getitem__(self, index):

        x_train_sample = np.uint8(self.x_train[index].reshape(28,28))

        if self.transform:
            x_train_pil = Image.fromarray(x_train_sample)
            x_train_np = np.array(self.transform(x_train_pil))
            x_train_tensor = torch.tensor(x_train_np[np.newaxis,:,:]).float().to(device)
        else:
            x_train_tensor = torch.tensor(x_train_sample[np.newaxis,:,:]).float().to(device)


        y_train_tensor = torch.tensor(self.y_train[index]).to(device)

        return x_train_tensor, y_train_tensor

    def __len__(self):
            return self.x_train.shape[0]

In [58]:
training_data = ASL_Dataset( "sign_mnist_train.csv", transform=transform)
validation_data = ASL_Dataset( "sign_mnist_valid.csv")

In [59]:
train_loader = DataLoader(dataset=training_data, batch_size=256, shuffle=True)
validation_loader = DataLoader(dataset=training_data, batch_size=256, shuffle=False)

### 2. Constructing a convolutional neural network



In [46]:
import torch.nn as nn

In [48]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 16, kernel_size = 3, stride = 1, padding = 1)
        self.conv2 = nn.Conv2d(16,32, kernel_size = 3, stride = 1, padding = 1)
        self.conv3 = nn.Conv2d(32,32, kernel_size = 3, stride = 1, padding = 1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2,2)
        self.fc = nn.Linear(32*7*7, 24)

    def forward(self, x):

        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)

        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)

        x = self.conv3(x)
        x = self.relu(x)

        x = x.view(-1, 32*7*7)
        x = self.fc(x)
  
        return x

In [60]:
convnet = ConvNet().to(device)

### 3. Loss function and Optimization method

In [61]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(convnet.parameters(), lr = 1e-3)

### 4. Training of a neural network

In [62]:
for epoch in range(20):
    loss_val = 0
    for itr, data in enumerate(train_loader):
        input, label = data
        
        optimizer.zero_grad()

        cost = convnet(input)
        loss = loss_function(cost,label)

        loss.backward()
        optimizer.step()  

        loss_val += loss.item()

    print("Epoch:", epoch+1, "  , Loss:", loss_val)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: 1   , Loss: 223.77333253622055
Epoch: 2   , Loss: 55.67685857415199
Epoch: 3   , Loss: 26.129411444067955
Epoch: 4   , Loss: 14.472962468862534
Epoch: 5   , Loss: 9.112371016293764
Epoch: 6   , Loss: 6.886278823018074
Epoch: 7   , Loss: 4.857309810817242
Epoch: 8   , Loss: 4.196681473404169
Epoch: 9   , Loss: 3.3717581406235695
Epoch: 10   , Loss: 2.5802329529542476
Epoch: 11   , Loss: 3.2704843361862004
Epoch: 12   , Loss: 2.9373330646194518
Epoch: 13   , Loss: 2.9639740055426955
Epoch: 14   , Loss: 1.9378956754226238
Epoch: 15   , Loss: 1.4056201614439487
Epoch: 16   , Loss: 2.3468199253547937
Epoch: 17   , Loss: 1.9360727994935587
Epoch: 18   , Loss: 1.2467894761066418
Epoch: 19   , Loss: 1.06072497140849
Epoch: 20   , Loss: 1.3304553089255933


### 5. Prediction and evaluation for a test set

In [64]:
pred_list = []
label_list = []
for itr, data in enumerate(validation_loader):
    inputs, labels = data
    pred_test = convnet(inputs)   
    pred_cate = torch.argmax(pred_test, axis = 1)
    
    pred_list += list(pred_cate.cpu())
    label_list += list(labels.cpu())  

accu = np.mean(np.array(pred_list) == np.array(label_list))
print("Test accuracy:", accu)

Test accuracy: 0.9959570205791295
